# Traveling Salesman Problem 

## Objective is to model the TSP as a Mixed Integer Problem model and use lazy constraints to identify solutions that are infeasible

A TSP setting can be as follows: for a given set of cities and the distances between each pair of them, the need is to find the shortest possible route that goes to each city once and returns to the origin city.

Here Euclidean distances are used, but the TSP model formulation is valid independent of the way in which the individual distances are determined,

Model Formulation 

### Sets and Indices 

$i, j \in Capitals $: represents indices and set of capital cities.

$\text{Pairings}= \{(i,j) \in Capitals \times Capitals \}$: Set of allowed pairings

$S \subset Capitals$: A subset of the set capital cities.

$G = (Capitals, Pairings)$: A graph where the set $Capitals$ defines the set of nodes and the set $Pairings$ defines the set of edges. 


### Parameters 


$d_{i, j} \in \mathbb{R}^+$: Distance from capital city $i$ to capital city $j$, for all $(i, j) \in Pairings$. 

The distance from capital city $i$ to capital city $j$ is assumed to be same as the distance from capital city $j$ to capital city $i$ thereby making $d_{i, j} = d_{j, i}$. For this reason, this TSP is a special case of **symmetric Traveling Salesman Problem.**

### Decision Variables 

$x_{i, j} \in \{0, 1\}$: This variable is equal to 1, if there is a travel step between city $i$ and city $j$. Otherwise, the decision variable is equal to zero.

### Objective Function
- **Shortest Route**. Minimize the total distance of the whole route. A route is a sequence of capital cities such that the salesperson visits each city only once and returns to the starting capital city.

\begin{equation}
\text{Min} \quad Z = \sum_{(i,j) \in \text{Pairings}}d_{i,j} \cdot x_{i,j}
\tag{0}
\end{equation}

### Constraints 

- **Symmetry Constraints**. For each edge $(i,j)$,it is important to ensure that the city capitals $i$ and $j$ are connected, if the former is visited immediately before or after visiting the latter.

\begin{equation}
x_{i, j} = x_{j, i} \quad \forall (i, j) \in Pairings
\tag{1}
\end{equation}

- **Entering and leaving a capital city**. For each capital city $i$,it is critical to ensure that this city is connected to two other cities. 

\begin{equation}
\sum_{(i,j) \in \text{Pairings}}x_{i,j} = 2 \quad \forall  i \in Capitals
\tag{2}
\end{equation}

- **Subtour elimination**. These constraints ensure that for any subset of cities $S$ of the set of $Capitals$, there is no cycle. That is, there is no route that visits all the cities in the subset and returns to the origin city.

\begin{equation}
\sum_{(i \neq j) \in S}x_{i,j} \leq |S|-1 \quad \forall  S \subset  Capitals
\tag{3}
\end{equation}


- **Note**. In general, if the number of cities of the TSP is $n$, then the possible number of routes is n\!.
Since there are an exponential number of constraints ($2^{n} - 2$) to eliminate cycles, we use lazy constraints to dynamically eliminate those cycles.

#### Implementing using Python 

The following libraries were imported:
* math : for mathematical functions
* itertools: for implementation of iterating blocks 
* folium: for creation of maps 
* gurobipy: Library to implement Grobi algorithms 

In [21]:
import json
# Reading capital names and coordinates of those capitals from the json file capitals.json
capitals_json=json.load(open('capitals.json'))
capitals=[]
coordinates={}
for state in capitals_json:
    if state not in ['AK','HI']:
        capital = capitals_json[state]['capital']
        capitals.append(capital)
        coordinates[capital]=(float(capitals_json[state]['lat']),float(capitals_json[state]['long']))

### Preprocessing 

In [22]:
import math
from itertools import combinations,product
## Computing pairwise distance. Note we are directly importing the methods of need so that we do not have to do itertolls.product() 
def distance (city1,city2):
    c1=coordinates[city1]
    c2=coordinates[city2]
    diff=(c1[0]-c2[0],c1[1]-c2[1])
    return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])

dist={(c1,c2):distance(c1,c2)for c1,c2 in product (capitals,capitals) if c1!=c2}


## The next function deals with lazy constraints so as to eliminate sub-tours


In [17]:
## Callback - use lazy constraints to eliminate sub-tours 
def subtourelimination(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys()
                             if vals[i, j] > 0.5)
        # finding the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < len(capitals):
            # add subtour elimination constr. for every pair of cities in subtour
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2))
                         <= len(tour)-1)

### Finding the shortest route


In [23]:
# Given a tuplelist of edges, find the shortest subtour

def subtour(edges):
    unvisited = capitals[:]
    cycle = capitals[:] # Dummy - guaranteed to be replaced
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle # New shortest subtour
    return cycle

### Deploying the model

In [24]:
import gurobipy as gp
from gurobipy import GRB


m = gp.Model()


vars = m.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='e')
for i, j in vars.keys():
    m.addConstr(vars[j, i] == vars[i, j])  # edge in opposite direction

# Constraints: two edges incident to each city

m.addConstrs(vars.sum(c, '*') == 2 for c in capitals)

# Optimize the model

m._vars = vars
m.Params.lazyConstraints = 1
m.optimize(subtourelimination)


Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 2304 rows, 2256 columns and 6768 nonzeros
Model fingerprint: 0x029d7617
Variable types: 0 continuous, 2256 integer (2256 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve removed 2256 rows and 1128 columns
Presolve time: 0.01s
Presolved: 48 rows, 1128 columns, 2256 nonzeros
Variable types: 0 continuous, 1128 integer (1128 binary)

Root relaxation: objective 3.223715e+02, 72 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  322.37153    0   12          -  322.37153      -     -    0s
     0     0  326.22035    0   18          -  326.22035      -     -    0s
     

In [25]:
# Retrieve solution

vals = m.getAttr('x', vars)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

tour = subtour(selected)
assert len(tour) == len(capitals)

In [26]:
# Map the solution

import folium

map = folium.Map(location=[40,-95], zoom_start = 4)

points = []
for city in tour:
  points.append(coordinates[city])
points.append(points[0])

folium.PolyLine(points).add_to(map)

map